In [1]:
import sys
import os
import numpy as np
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

## Configure the model training policy


In [2]:
model_development.configure_training_policy()

## Load and preprocess data

In [3]:
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv')
print([cell_images.shape, len(cell_labels), len(cell_types)])

# @interact(n = (0, cell_images.shape[0] - 1))
# def display_data(n = 0):
#     fig, axs = plt.subplots()
#     axs.imshow(cell_images[n], cmap = 'gray')
#     axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
#     return None

[(2624, 300, 300), 2624, 2624]


## Prepare data for model training

In [4]:
mono_good = data_handling.query_data_by_labels_and_types(data = cell_images, labels = cell_labels, types = cell_types,
    filter_by_labels = 0.0, filter_by_types = 'mono')
poly_good = data_handling.query_data_by_labels_and_types(data = cell_images, labels = cell_labels, types = cell_types,
    filter_by_labels = 0.0, filter_by_types = 'poly')
X = np.concatenate((mono_good, poly_good), axis = 0)
X = np.stack((X,) * 3, axis = -1)

train_test_split = int(0.85 * X.shape[0])
X_train, X_test = X[: train_test_split], X[train_test_split :]
print([X_train.shape, X_test.shape])

[(1281, 300, 300, 3), (227, 300, 300, 3)]


## Train classification model (first phase)

In [5]:
optimizer = model_development.create_optimizer('nadam')
cae_vgg19 = \
    model_development.cae_VGG19(input_shape = (300, 300, 3), weights = 'imagenet', freeze_convolutional_base = True, display_model_information = False)

history, training_time = model_development.train_reconstruction_model(1, cae_vgg19, optimizer = optimizer, model_name = 'CAE_VGG19', version = '1.00.00_fs',
    X = X_train, Y = X_train, 
    metric_to_monitor = 'val_loss', n_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
68/68 [==============================] - ETA: 0s - loss: 0.4021
Epoch 1: val_loss improved from inf to 0.26576, saving model to ../models/weights\CAE_VGG19_1.00.00_fs.hdf5
68/68 [==============================] - 92s 1s/step - loss: 0.4021 - val_loss: 0.2658
Epoch 2/200
68/68 [==============================] - ETA: 0s - loss: 0.1987
Epoch 2: val_loss improved from 0.26576 to 0.20845, saving model to ../models/weights\CAE_VGG19_1.00.00_fs.hdf5
68/68 [==============================] - 80s 1s/step - loss: 0.1987 - val_loss: 0.2085
Epoch 3/200
68/68 [==============================] - ETA: 0s - loss: 0.1731
Epoch 3: val_loss did not improve from 0.20845
68/68 [==============================] - 79s 1s/step - loss: 0.1731 - val_loss: 0.2567
Epoch 4/200
68/68 [==============================] - ETA: 0s - loss: 0.1766
Epoch 4: val_loss improved from 0.20845 to 0.18074, saving model to ../models/weights\CAE_VGG19_1.00.00_fs.hdf5
68/68 [==============================] - 81s 1s/step - l

## Train classification model (second phase)

In [6]:
optimizer = model_development.create_optimizer('adam')
history, training_time = \
    model_development.train_reconstruction_model(2, '../models/weights/CAE_VGG19_1.00.00_fs.hdf5', optimizer = optimizer, 
    model_name = 'CAE_VGG19', version = '1.00.00_ss',
    X = X_train, Y = X_train, 
    metric_to_monitor = 'val_loss', n_of_epochs = 200, batch_size = 8, validation_split_ratio = 0.15)

Epoch 1/200
136/136 [==============================] - ETA: 0s - loss: 0.0997
Epoch 1: val_loss improved from inf to 0.17042, saving model to ../models/weights\CAE_VGG19_1.00.00_ss.hdf5
136/136 [==============================] - 148s 1s/step - loss: 0.0997 - val_loss: 0.1704
Epoch 2/200
136/136 [==============================] - ETA: 0s - loss: 0.0930
Epoch 2: val_loss did not improve from 0.17042
136/136 [==============================] - 139s 1s/step - loss: 0.0930 - val_loss: 0.1715
Epoch 3/200
136/136 [==============================] - ETA: 0s - loss: 0.0903
Epoch 3: val_loss improved from 0.17042 to 0.16637, saving model to ../models/weights\CAE_VGG19_1.00.00_ss.hdf5
136/136 [==============================] - 138s 1s/step - loss: 0.0903 - val_loss: 0.1664
Epoch 4/200
136/136 [==============================] - ETA: 0s - loss: 0.0891
Epoch 4: val_loss improved from 0.16637 to 0.16140, saving model to ../models/weights\CAE_VGG19_1.00.00_ss.hdf5
136/136 [==============================